In [32]:
from sympy.physics.units import temperature
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments
import torch
checkpoint_path = "Qwen/checkpoints/checkpoint-1500"
tokenizer = AutoTokenizer.from_pretrained(checkpoint_path)
model = AutoModelForCausalLM.from_pretrained(
    checkpoint_path,
    torch_dtype=torch.bfloat16,
    device_map="cuda",
    attn_implementation="flash_attention_2"
)
model.bfloat16()

Qwen3ForCausalLM(
  (model): Qwen3Model(
    (embed_tokens): Embedding(151936, 1024)
    (layers): ModuleList(
      (0-27): 28 x Qwen3DecoderLayer(
        (self_attn): Qwen3Attention(
          (q_proj): Linear(in_features=1024, out_features=2048, bias=False)
          (k_proj): Linear(in_features=1024, out_features=1024, bias=False)
          (v_proj): Linear(in_features=1024, out_features=1024, bias=False)
          (o_proj): Linear(in_features=2048, out_features=1024, bias=False)
          (q_norm): Qwen3RMSNorm((128,), eps=1e-06)
          (k_norm): Qwen3RMSNorm((128,), eps=1e-06)
        )
        (mlp): Qwen3MLP(
          (gate_proj): Linear(in_features=1024, out_features=3072, bias=False)
          (up_proj): Linear(in_features=1024, out_features=3072, bias=False)
          (down_proj): Linear(in_features=3072, out_features=1024, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen3RMSNorm((1024,), eps=1e-06)
        (post_attention_layernorm): Qwe

In [63]:
temp = [
    {"role": "user", "content": "Generate SVG from description 'Lavender desert stretching endlessly under moonlight'. Think about the scene firstю. Use 'path' teg more often, add more gradients. Make the objects bigger and inside the scene. "},
]
text = tokenizer.apply_chat_template(
    temp,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=True,
)

In [64]:
print(text)

<|im_start|>user
Generate SVG from description 'Lavender desert stretching endlessly under moonlight'. Think about the scene firstю. Use 'path' teg more often, add more gradients. Make the objects bigger and inside the scene. <|im_end|>
<|im_start|>assistant



In [76]:
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)
model_inputs
generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=1000,
    temperature=0.5,
)

In [77]:
output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist()
generated_ids.shape

torch.Size([1, 1052])

In [78]:
# parsing thinking content
try:
    # rindex finding 151668 (</think>)
    index = len(output_ids) - output_ids[::-1].index(151668)
except ValueError:
    index = 0

thinking_content = tokenizer.decode(output_ids[:index], skip_special_tokens=True).strip("\n")
content = tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip("\n")

print("thinking content:", thinking_content)
print("content:", content)

thinking content: <think>
Okay, let's tackle this SVG request. The user wants an SVG based on the description: "Lavender desert stretching endlessly under moonlight." First, I need to visualize the scene. A lavender desert under the moon would have elements like the horizon, the desert grass, maybe some stars or the moon.

Since they mentioned using paths more and adding gradients, I should start by creating a base structure. The desert can be represented with a path or a group. Maybe a rectangle for the desert area. Then, I need to add elements like the moon and stars. The moon could be a circle or a polygon. For gradients, maybe a linear gradient for the desert and a radial one for the moon.

Objects inside the scene should be bigger and more detailed. So, I'll use paths for each element. The desert grass could be a path with a gradient. The stars might be another path with a gradient. The horizon could be another path. Also, adding elements like the sun or the stars to the scene.

W

<svg width="800" height="600" xmlns="http://www.w3.org/2000/svg">
  <!-- Background and sky -->
  <defs>
    <linearGradient id="desertGradient" x1="0%" y1="0%" x2="100%">
      <stop offset="0%" stop-color="#E6E6FA" />
      <stop offset="100%" stop-color="#D8D8DC" />
    </linearGradient>

    <linearGradient id="moonGradient" x1="0%" y1="0%" x2="100%">
      <stop offset="0%" stop-color="#0087ff" />
      <stop offset="100%" stop-color="#0087ff" />
    </linearGradient>
  </defs>

  <!-- Moon -->
  <circle cx="400" cy="450" r="100" fill="#0087ff" />

  <!-- Dunes (pale purple gradient) -->
  <polygon points="500,300 580,300 650,300" fill="url(#desertGradient)" />

  <!-- Valleys (indigo shadow) -->
  <polygon points="400,350 450,350" fill="url(#indigoGradient)" />

  <!-- Plant cluster (violet blooms) -->
  <polygon points="350,330 410,330 460,330" fill="#6b8e23" />

  <!-- Plant bloom (violet) -->
  <polygon points="380,335 420,335" fill="#7e8824" />

  <!-- Dunes (silver-blue highlights) -->
  <polygon points="460,330 500,300 580,300" fill="url(#silverBlueGradient)" />

  <!-- Plant (violet) -->
  <polygon points="420,335 460,330" fill="#7b8a24" />

  <!-- Plant (violet) -->
  <polygon points="370,335 400,335" fill="#7d8824" />
</svg>

##  Qwen-0.6B temp 0.7

<svg width="1000" height="600" xmlns="http://www.w3.org/2000/svg">
  <!-- Background -->
  <rect width="100%" height="100%" fill="#FFB88B" />

  <!-- Desert Scene -->
  <rect x="0" y="0" width="1000" height="400" fill="#f4d100" />
  <polygon points="100,100 300,100 300,300 500,300 500,100" fill="#f4d100" />
  <polygon points="200,200 400,200 400,400 600,400 600,200" fill="#f4d100" />

  <!-- Moonlight Effect -->
  <circle cx="500" cy="300" r="100" fill="#008000" />
  <circle cx="400" cy="350" r="80" fill="#008000" />
  <circle cx="600" cy="300" r="80" fill="#008000" />

  <!-- Stars -->
  <circle cx="450" cy="250" r="50" fill="#FF0000" />
  <circle cx="550" cy="280" r="45" fill="#FF0000" />
  <circle cx="650" cy="320" r="40" fill="#FF0000" />

  <!-- Lavender Flowers -->
  <circle cx="200" cy="200" r="40" fill="#FFB88B" />
  <circle cx="300" cy="250" r="30" fill="#FFB88B" />
  <circle cx="400" cy="300" r="25" fill="#FFB88B" />
</svg>

##  Qwen-0.6B fine tuned temp 0.7

<svg width="600" height="400" xmlns="http://www.w3.org/2000/svg">
  <!-- Background gradient for the lavender desert -->
  <defs>
    <pattern id="moonlit-desert" width="200" height="200" patternUnits="userSpace">
      <rect x="0" y="0" width="200" height="200" fill="#2e6732"/>
      <path d="M 100 100 v-50 h-20 v-30 h-16 v-20 h30 v-10 h-18 v-12 h-16 v-30 h20 v100 z"/>
    </pattern>
  </defs>

  <!-- Lavender desert shape -->
  <rect x="0" y="0" width="600" height="400" fill="url(#moonlit-desert)" />

  <!-- Moonlight effect -->
  <circle cx="300" cy="250" r="150" fill="#809c9b" />
  <circle cx="200" cy="200" r="120" fill="#388056" />
</svg>

##  Qwen-0.6B fine tuned temp 1

<svg width="600" height="400" xmlns="http://www.w3.org/2000/svg">
  <!-- Title and Description -->
  <text x="300" y="35" font-size="24" font-family="Arial" text-anchor="middle" dominant-baseline="middle">
    Lavender desert stretching endlessly under moonlight
  </text>

  <!-- Desert Background -->
  <rect x="0" y="0" width="600" height="400" fill="#e0f7e0" />

  <!-- Path for the Lavender Ridge -->
  <path d="M 300,30 v40 h -30 v-3 H 270 V 330 h -20 V 325 H 200 v -4 l -32, 1 -40, 2 -86, -1 32 -4 z" fill="#90d25d" />

  <!-- Moonlight Shadow -->
  <circle cx="330" cy="235" r="16" fill="#7a9d70" />

  <!-- Lavender Flowers and Path -->
  <text x="280" y="270" font-size="16" fill="#96b799">Lavender path</text>
  <text x="210" y="280" font-size="14" fill="#90c66b">Flowers</text>

  <!-- Additional Details -->
  <text x="290" y="360" font-size="14" fill="#6c8667">Stretching under</text>
</svg>

##  Qwen-0.6B temp 1

<svg width="400" height="300" xmlns="http://www.w3.org/2000/svg">
  <!-- Background - Lavender Desert -->
  <rect x="0" y="0" width="400" height="300" fill="#e0f2fd" />

  <!-- Moonlit Desert -->
  <circle cx="200" cy="150" r="80" fill="rgba(230, 153, 255, 0.8)" />
  <circle cx="180" cy="120" r="40" fill="rgba(230, 153, 255, 0.8)" />
  <circle cx="210" cy="160" r="40" fill="rgba(230, 153, 255, 0.8)" />
  <line x1="180" y1="100" x2="180" y2="220" stroke="white" stroke-width="2" />
  <line x1="220" y1="100" x2="220" y2="220" stroke="white" stroke-width="2" />

  <!-- Wind Pattern -->
  <line x1="180" y1="100" x2="180" y2="220" />
  <line x1="220" y1="100" x2="220" y2="220" />

  <!-- Grass and Stars -->
  <polygon points="180,100 210,160 150,200" fill="#5b5b5b" />
  <polygon points="220,100 250,160 210,200" fill="#7b7b7b" />
  <polygon points="150,100 180,160 130,200" fill="#4b4b4b" />

  <!-- Additional Moonlight Elements -->
  <circle cx="200" cy="150" r="30" fill="rgba(230, 153, 255, 0.5)" />
</svg>

<svg width="240" height="240" xmlns="http://www.w3.org/2000/svg">
  <!-- Background: Tranquil Lavender Sky -->
  <defs>
    <g id="sky-gradient">
      <filter id="shade" x="0%" y="0%" width="100%" height="100%">
        <!-- Light lavender filter -->
        <feDropShadow cx="-22px" cy="10px" from="4px"/>
        <feFuncRune id="lumid-grad" x="1" y="-30%">
          <rect width="153%" from="@#" />
          <rect width="247%" from="#95CEA8" />
        </feFuncRune>
      </filter>
    </g>
  </defs>

  <!-- Main Features: Desert, Moon, Stars, Elements -->
  <!-- Moon -->
  <rect width="140" height="120" fill="#EAF1FF" />

  <!-- Moonlit Scene - Moon Circle & Stars */
  <circle cx="35" cy="280" r="70"/>
  <g fill="white">
    <circle cx="68" cy="110" r="5"/> <!-- Star in foreground -->
    <circle cx="75" cy="205" r="4"/>
  <!-- Stars - 10 to 36, each at specific coordinates --> 
    
  <!-- Desert Spread - Lapis lazuli, stretching -->
  <!-- Add more details with circles here, e.g., -->
  <circle cx="71" cy="71"> <!-- 8th element with lapis --> </circle>
  <rect fill="url(#lapis)" opacity="1" width="66%"></rect>

  <!-- Optional Stars or Clouds to Add Depth -->
  <!-- Clouds, Mountains as decorative details -->
  <!-- More elements go into specific `<circle />`, `rectangles`, etc. if specified -->

</svg>

<svg xmlns="http://www.w3.org/2000/svg" width="200" height="150" viewBox="0 0 200 150">
    <!-- Background gradient -->
    <defs>
        <linearGradient id="bgGradient" x1="0%" y1="0%" x2="100%" y2="100%">
            <stop offset="0%" stop-color="#E6E6FA"/>
            <stop offset="100%" stop-color="#B0C4DE"/>
        </linearGradient>
    </defs>
    
    <rect width="100%" height="100%" fill="url(#bgGradient)"/>
    
    <!-- Moon -->
    <circle cx="160" cy="30" r="15" fill="#F0F8FF" opacity="0.9"/>
    
    <!-- Dunes -->
    <polygon points="0,120 200,80 200,150 0,150" fill="#DEB887"/>
    
    <!-- Lavender bush -->
    <g transform="translate(80 100)">
        <path d="M0 0 Q10 -20 20 0 Q30 20 40 0 Q50 -20 60 0 L60 40 L0 40 Z" fill="#9370DB"/>
        <ellipse cx="30" cy="35" rx="25" ry="10" fill="#6B8E23"/>
    </g>
    
    <!-- Text label -->
    <text x="100" y="30" font-family="Arial" font-size="12" fill="#4B0082" 
          text-anchor="middle" font-weight="bold">
        Lavender Desert
    </text>
</svg>

<svg xmlns="http://www.w3.org/2000/svg" clip-rule="evenodd" image-rendering="optimizeQuality" shape-rendering="geometricPrecision" text-rendering="geometricPrecision" viewBox="0 0 96 96"><path fill="#e3cac5" d="M0 0h24l-4 2q6 3 2 9l2 3-4 6v11l2 5 9 2H12q-6-3-12-2z"/><path fill="#e6cfca" d="M24 0h18l16 3 9-1 3 2-1 2q6 7-2 5 0 2 3 2-5 2-5 6v6q3 4 0 7v4l6 2-14 1-35-3-2-5V20l4-6-2-3q4-6-2-9z"/><path fill="#e5ccc7" d="M42 0h54v38H71l-6-2v-4q3-3 0-7v-6q0-4 5-6-3 0-3-2 8 2 2-5l1-2-3-2-9 1z"/><path fill="#f2efeb" d="M47 18q5-1 3 4-6 0-3-4"/><path fill="#d18475" d="M12 39v-1h19l26 1v2H34q-10-2-19-1-1-2-3-1"/><path fill="#cd9490" d="M71 38h25v2l-8-1v3l-22-2-9 1v-2z"/><path fill="#ca9fa1" d="M96 53v16l-9 9q-2 0-1 3l-10 1 1 1q-3 2-3 5l-6-1-2 1v3q-3-3-7-3-2 0-1 3l-7-5-1 1q-1-3 1-5l-1-1q-2 0-2 3l-1-4q-5 5-12 6l-3 3h-3q-4 1 0 4-5-1-8 1l2 2H0V63q2 0 3 2l1-2q4 2 4 7 3 3 0 4 3 5 4-1l7 3 5-3 2 2 15-5 3 6 1-3 1 4q2-1 3 2 4-4 11-6-1 2 1 3l4-2 6 1q8-4 18-6l-4-2q5-2-1-2 3-9 12-12"/><path fill="#c7999a" d="M0 36q6-1 12 2v1H6l-2 5q0 2 2 3-2 1-2 4l1 9 11-2q8-1 15 2 3-2 8-1l1 2 4-4h3q3 2-1 3l2 5q0-3 2-3h1l-3 4 18-1q2-3 6-2-6-1-1-4l9-2-6-2h5q5-2 11-1l-2-6h4l-1-6h-3v-3l8 1v13q-9 3-12 12 6 0 1 2l4 2q-10 2-18 6l-6-1-4 2q-2-1-1-3-7 2-11 6-1-3-3-2l-1-4-1 3-3-6-15 5-2-2-5 3-7-3q-1 6-4 1 3-1 0-4 0-5-4-7l-1 2q-1-2-3-2z"/><path fill="#6f5175" d="m79 51 1 2q-2 0-1 2h-5l-1-2v-3l4-1z"/><path fill="#342459" d="M79 51q-2 4-5 1 2-2 5-1"/><path fill="#c29192" d="M66 50q4-1 7 3l1 2 6 2-9 2q-5 3 1 4-4-1-6 2l-18 1 3-4h-1q-2 0-2 3l-2-5q4-1 1-3h-3l-4 4-1-2q-5-1-8 1-7-3-15-2-1-3 1-5a257 257 0 0052-2z"/><path fill="#cd9387" d="M47 48h9l1 1 9 1 3 1-14 2z"/><path fill="#be8c96" d="M88 42h3l1 6h-4l2 6q-6-1-11 1-1-2 1-2l1-5q-9-3-16-1-6-3-9-1-10-4-21-3l-1-2h23l9-1z"/><path fill="#684a6c" d="M56 47q2-1 3 1l-2 1z"/><path fill="#ae7d92" d="m80 53-1-2-2-2-4 1v3q-3-4-7-3l-9-1 2-1q-1-2-3-1v-1q3-2 9 1 7-2 16 1z"/><path fill="#d89a84" d="M35 43q11-1 21 3v2h-9l-12-3q-9-2 0-2"/><path fill="#bc8e98" d="M12 39q2-1 3 1-4-1-1 2l8 3q-4 2 0 3 3-2 8 1l1-2q2 3 6 1-2-1-2-3l12 3 8 5H17q-2 2-1 5L5 60l-1-9q0-3 2-4-2-1-2-3l2-5z"/><path fill="#b4879a" d="M15 40q9-1 19 1l1 2q-9 0 0 2 0 2 2 3-4 2-6-1l-1 2q-5-3-8-1-4-1 0-3l-8-3q-3-3 1-2"/><path fill="#7e5e86" d="M26 44q2-1 3 1-3 1-3-1"/><path fill="#cda4a3" d="M96 69v27H23l-2-2q3-2 8-1-4-3 0-4h3l3-3q7-1 12-6l1 4q0-3 2-3l1 1q-2 2-1 5l1-1 7 5q-1-3 1-3 4 0 7 3v-3l2-1 6 1q0-3 3-5l-1-1 10-1q-1-3 1-3z"/></svg>